<a href="https://colab.research.google.com/github/chyj0805/Project_2/blob/main/colabs/Yujie/data_fetches%26future_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Fetches

In [ ]:
!pip install python-dotenv

# Funcation need to be define before using

In [ ]:
# Integrating the environment variable loading and API key retrieval into the function

from dotenv import load_dotenv
import requests
import pandas as pd
import os


def get_stock_historical_data_with_MACD(stock_symbol, interval="daily", series_type="close"):
    # Load the environment variable file and get API key
    load_dotenv()
    api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")

    if api_key is None:
        raise ValueError("Alpha Vantage API key is not set in the environment.")

    base_url = "https://www.alphavantage.co/query"

    # Fetch Stock Historical Data
    stock_params = {
        "function": "TIME_SERIES_DAILY",
        "symbol": stock_symbol,
        "apikey": api_key,
        "outputsize": "full",
        "datatype": "json"
    }

    response = requests.get(base_url, params=stock_params)
    stock_data = response.json()

    if "Time Series (Daily)" not in stock_data:
        print("Error fetching stock data. Check your API key and stock symbol.")
        return None

    time_series = stock_data["Time Series (Daily)"]
    stock_list = []

    for date, details in time_series.items():
        row = {
            "Date": date,
            "Open": details["1. open"],
            "High": details["2. high"],
            "Low": details["3. low"],
            "Close": details["4. close"],
            "Volume": details["5. volume"]
        }
        stock_list.append(row)

    stock_df = pd.DataFrame(stock_list)
    stock_df["Date"] = pd.to_datetime(stock_df["Date"])
    stock_df = stock_df.sort_values(by="Date")

    # Fetch MACD Data
    macd_params = {
        "function": "MACD",
        "symbol": stock_symbol,
        "interval": interval,
        "series_type": series_type,
        "apikey": api_key,
        "datatype": "json"
    }

    response = requests.get(base_url, params=macd_params)
    macd_data = response.json()

    if "Technical Analysis: MACD" not in macd_data:
        print("Error fetching MACD data. Check your API key and stock symbol.")
        return None

    macd_series = macd_data["Technical Analysis: MACD"]
    macd_list = []

    for date, details in macd_series.items():
        row = {
            "Date": date,
            "MACD_Signal": details["MACD_Signal"],
            "MACD_Hist": details["MACD_Hist"],
            "MACD": details["MACD"],
        }
        macd_list.append(row)

    macd_df = pd.DataFrame(macd_list)
    macd_df["Date"] = pd.to_datetime(macd_df["Date"])
    macd_df = macd_df.sort_values(by="Date")

    # Merge DataFrames
    merged_df = pd.merge(stock_df, macd_df, on="Date", how="inner")

    return merged_df

# Example usage:
# final_df = get_stock_historical_data_with_MACD("AAPL")
# print(final_df.head())


In [ ]:
IYE = get_stock_historical_data_with_MACD("IYE")

In [ ]:
IYE

,Date,Open,High,Low,Close,Volume,MACD_Signal,MACD_Hist,MACD
0,2000-08-03,49.1900,49.1900,48.5000,48.5300,1500,-0.2024,0.0831,-0.1194
1,2000-08-04,47.7500,48.5300,47.7500,48.5300,2300,-0.1809,0.0859,-0.0950
2,2000-08-07,48.8800,49.2800,48.8800,49.1300,1000,-0.1578,0.0925,-0.0652
3,2000-08-08,49.0300,49.3100,49.0300,49.3100,400,-0.1339,0.0956,-0.0383
4,2000-08-09,50.0600,50.4100,50.0000,50.2200,7200,-0.1076,0.1053,-0.0023
...,...,...,...,...,...,...,...,...,...
5801,2023-08-25,45.8800,46.2900,45.5100,46.0500,455569,0.6129,-0.1964,0.4166
5802,2023-08-28,46.2100,46.7200,46.1140,46.3900,361019,0.5717,-0.1649,0.4068
5803,2023-08-29,46.4800,46.6125,46.0700,46.5600,386931,0.5390,-0.1309,0.4081
5804,2023-08-30,46.7300,46.9150,46.6100,46.8100,181727,0.5161,-0.0917,0.4244


In [ ]:
def get_federal_funds_rate(interval="daily"):
    # Load the API key from environment variables
    load_dotenv()
    api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")

    if api_key is None:
        raise ValueError("Alpha Vantage API key is not set in the environment.")

    # Fetch Federal Funds Rate data
    url = f'https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval={interval}&apikey={api_key}'
    response = requests.get(url)
    data = response.json()

    if 'data' not in data:
        print("Error fetching FEDERAL_FUNDS_RATE data. Check your API key.")
        return None

    # Extract Federal Funds Rate data
    fed_time_series_data = data['data']
    fed_dates = [entry['date'] for entry in fed_time_series_data]
    fed_rates = [float(entry['value']) for entry in fed_time_series_data]

    # Create a DataFrame for Federal Funds Rate
    fed_df = pd.DataFrame({'Federal Funds Rate': fed_rates}, index=fed_dates)
    fed_df.index = pd.to_datetime(fed_df.index)

    # Sort the DataFrame by date index from oldest to newest
    fed_df.sort_index(ascending=True, inplace=True)

    return fed_df

# Example usage:
federal_funds_rate_df = get_federal_funds_rate()
if federal_funds_rate_df is not None:
    print(federal_funds_rate_df.head())


            Federal Funds Rate
1954-07-01                1.13
1954-07-02                1.25
1954-07-03                1.25
1954-07-04                1.25
1954-07-05                0.88


In [ ]:
from dotenv import load_dotenv
import requests
import pandas as pd
import os

def get_combined_data(stock_symbol, interval="daily", series_type="close"):
    # Load the environment variable file and get API key
    load_dotenv()
    api_key = os.environ.get("ALPHA_VANTAGE_API_KEY")

    if api_key is None:
        raise ValueError("Alpha Vantage API key is not set in the environment.")

    # Fetch Stock and MACD Data
    stock_macd_df = get_stock_historical_data_with_MACD(stock_symbol, interval, series_type)  # Assuming this function is already defined

    # Fetch Federal Funds Rate Data
    rate_url = f'https://www.alphavantage.co/query?function=FEDERAL_FUNDS_RATE&interval={interval}&apikey={api_key}'
    rate_data = requests.get(rate_url).json()

    if 'data' not in rate_data:
        print("Error fetching FEDERAL_FUNDS_RATE data. Check your API key.")
        return None

    fed_dates = [entry['date'] for entry in rate_data['data']]
    fed_rates = [float(entry['value']) for entry in rate_data['data']]
    fed_df = pd.DataFrame({'Federal Funds Rate': fed_rates}, index=fed_dates)
    fed_df.index = pd.to_datetime(fed_df.index)
    fed_df.sort_index(ascending=True, inplace=True)

    # Fetch Unemployment Data
    job_url = f'https://www.alphavantage.co/query?function=UNEMPLOYMENT&apikey={api_key}'
    job_data = requests.get(job_url).json()

    if 'data' not in job_data:
        print("Error fetching UNEMPLOYMENT data. Check your API key.")
        return None

    job_dates = [entry['date'] for entry in job_data['data']]
    job_rates = [float(entry['value']) for entry in job_data['data']]
    job_df = pd.DataFrame({'Unemployment Rate': job_rates}, index=job_dates)
    job_df.index = pd.to_datetime(job_df.index)
    job_df = job_df.resample('D').ffill()

    # Merge all DataFrames
    stock_macd_df['Date'] = pd.to_datetime(stock_macd_df['Date'])
    combined_df = pd.merge(stock_macd_df, fed_df, left_on='Date', right_index=True, how='left')
    combined_df = pd.merge(combined_df, job_df, left_on='Date', right_index=True, how='left')

    # Drop rows with NaN values
    combined_df.dropna(inplace=True)

    return combined_df

# Assuming the get_stock_historical_data_with_MACD function is defined as before
final_df = get_combined_data("AAPL")
if final_df is not None:
    print(final_df.head())


        Date      Open      High       Low     Close   Volume MACD_Signal  \
0 1999-12-17  100.8700  102.0000   98.5000  100.0000  4419700      0.0418   
1 1999-12-20   99.5600   99.6200   96.6200   98.0000  2535600      0.0377   
2 1999-12-21   98.1900  103.0600   97.9400  102.5000  2746400      0.0344   
3 1999-12-22  102.8700  104.5600   98.7500   99.9400  2920300      0.0313   
4 1999-12-23  101.8100  104.2500  101.0600  103.5000  2049400      0.0290   

  MACD_Hist    MACD  Federal Funds Rate  Unemployment Rate  
0   -0.0175  0.0243                5.40                4.0  
1   -0.0165  0.0211                5.54                4.0  
2   -0.0132  0.0211                5.52                4.0  
3   -0.0120  0.0193                5.45                4.0  
4   -0.0092  0.0198                5.40                4.0  


In [ ]:
IYE_df = final_df
IYE_df

,Date,Open,High,Low,Close,Volume,MACD_Signal,MACD_Hist,MACD,Federal Funds Rate,Unemployment Rate
0,1999-12-17,100.8700,102.0000,98.5000,100.0000,4419700,0.0418,-0.0175,0.0243,5.40,4.0
1,1999-12-20,99.5600,99.6200,96.6200,98.0000,2535600,0.0377,-0.0165,0.0211,5.54,4.0
2,1999-12-21,98.1900,103.0600,97.9400,102.5000,2746400,0.0344,-0.0132,0.0211,5.52,4.0
3,1999-12-22,102.8700,104.5600,98.7500,99.9400,2920300,0.0313,-0.0120,0.0193,5.45,4.0
4,1999-12-23,101.8100,104.2500,101.0600,103.5000,2049400,0.0290,-0.0092,0.0198,5.40,4.0
...,...,...,...,...,...,...,...,...,...,...,...
5916,2023-06-26,186.8300,188.0500,185.2300,185.2700,48088661,3.5274,-0.0194,3.5080,5.07,3.6
5917,2023-06-27,185.8900,188.3900,185.6700,188.0600,50730846,3.5365,0.0364,3.5729,5.07,3.6
5918,2023-06-28,187.9300,189.9000,187.6000,189.2500,51216801,3.5648,0.1131,3.6779,5.07,3.6
5919,2023-06-29,189.0800,190.0700,188.9400,189.5900,46347308,3.6009,0.1444,3.7453,5.07,3.6


# ydata-profilling

In [ ]:
pip install ydata-profiling

In [ ]:
from pandas_profiling import ProfileReport

IYE_profile = ProfileReport(IYE, title="IYE Historical Data Profiling Report")


<ipython-input-12-cc1e14cbb6b9>:1: DeprecationWarning: `import pandas_profiling` is going to be deprecated by April 1st. Please use `import ydata_profiling` instead.
  from pandas_profiling import ProfileReport


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
IYE_profile.to_file("IYE_Report.html")

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
IYE_profile